# Segmenting and Clustering Neighborhoods in London

In [1]:
# web scrapping
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
# Looks like not all websites are scrappable
page = requests.get('https://en.wikipedia.org/wiki/List_of_areas_of_London')
# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of areas of London - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"518c326d-b4c6-4939-be04-36a861f07920","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_areas_of_London","wgTitle":"List of areas of London","wgCurRevisionId":963608799,"wgRevisionId":963608799,"wgArticleId":11915713,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Use dmy dates from August 2015","Use British English from August 2015","Lists of coord

In [3]:
#match = soup.title.text
table = soup.find("table",class_='wikitable sortable')
#print(table)
tbody = table.find('tbody')
trs = tbody.find_all('tr')

data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 0):
            data_table.append(td.text.strip())
        count=count+1;
df2 = pd.DataFrame(data_table)
df = pd.DataFrame(data_table,columns=['Location'])


In [4]:
df.head()

,Location
0,Abbey Wood
1,Acton
2,Addington
3,Addiscombe
4,Albany Park


In [5]:

data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 1):
            data_table.append(td.text.strip())
        count=count+1;
df[['Borough']] = pd.DataFrame(data_table)

data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 2):
            data_table.append(td.text.strip())
        count=count+1;
df[['Post town']] = pd.DataFrame(data_table)

data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 3):
            data_table.append(td.text.strip())
        count=count+1;
df[['Postcode district']] = pd.DataFrame(data_table)

data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 4):
            data_table.append(td.text.strip())
        count=count+1;
df[['Dial code']] = pd.DataFrame(data_table)

data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 5):
            data_table.append(td.text.strip())
        count=count+1;
df[['OS grid ref']] = pd.DataFrame(data_table)




In [6]:
df.head()

,Location,Borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [7]:
df['Borough'] = df['Borough'].replace(regex={r'\[[1-9][0-9]]':''})


df.head()

,Location,Borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [8]:
df_proc = df[['Postcode district', 'Borough', 'Location']]

df_proc.rename(columns={"Postcode district": "Postcode", 
                        "Location":"Neighbourhood"},
               inplace=True)
df_proc.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postcode,Borough,Neighbourhood
0,SE2,"Bexley, Greenwich [7]",Abbey Wood
1,"W3, W4","Ealing, Hammersmith and Fulham[8]",Acton
2,CR0,Croydon[8],Addington
3,CR0,Croydon[8],Addiscombe
4,"DA5, DA14",Bexley,Albany Park


In [9]:
df_proc2 = df_proc.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_proc2.head()

,Postcode,Borough,Neighbourhood
0,BR1,Bromley,"Bromley, Plaistow, Sundridge, Widmore (also Wi..."
1,BR1,Lewisham,Downham
2,BR2,Bromley,"Hayes, Keston, Leaves Green, Southborough"
3,BR3,Bromley,"Bickley, Bromley Common, Eden Park, Elmers End"
4,"BR3, SE20",Bromley,Beckenham


In [10]:
#https://www.townslist.co.uk

df_data_1 = pd.read_csv('uk-towns-sample.csv')
df_data_1.head()
#df = df_data_1[df_data_1["county"]=="Greater London"]
df = df_data_1[df_data_1["country"]=="England"]
df.head()

,id,name,county,country,grid_reference,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type
0,1,Aaron's Hill,Surrey,England,SU957435,495783,143522,51.18291,-0.63098,78,GU7 2,Waverley District,South East,Suburban Area
1,2,Abbas Combe,Somerset,England,ST707226,370749,122688,51.00283,-2.41825,91,BA8 0,South Somerset District,South West,Village
2,3,Abberley,Worcestershire,England,SO744675,374477,267522,52.30522,-2.37574,152,WR6 6,Malvern Hills District,West Midlands,Village
3,4,Abberton,Essex,England,TM006190,600637,219093,51.83440,0.91066,44,CO5 7,Colchester District,Eastern,Village
4,5,Abberton,Worcestershire,England,SO995534,399538,253477,52.17955,-2.00817,68,WR10 2,Wychavon District,West Midlands,Hamlet


In [11]:
df.drop_duplicates(subset='name', keep='first', inplace=True)
df = df.head(100)
df.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(100, 14)

In [12]:
# plot on folium the neighborhoods of London, importing the libraries
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
# commenting off since it's already completed
import folium # map rendering library

In [13]:
latitude = 51.507350
longitude = -0.127756

# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['country'], df['name']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [14]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605'

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
london_venues = getNearbyVenues(names=df['name'],latitudes=df['latitude'],longitudes=df['longitude'])

london_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbas Combe,51.00283,-2.41825,Templecombe Railway Station (TMC),51.001564,-2.417480,Train Station
1,Abbas Combe,51.00283,-2.41825,The Templars Retreat,51.001755,-2.415475,Pub
2,Abbas Combe,51.00283,-2.41825,interim4transformation,51.000759,-2.414497,Shipping Store
3,Abberley,52.30522,-2.37574,Abberley,52.304917,-2.374229,Scenic Lookout
4,Abberton,51.83440,0.91066,Lucy's Health & Beauty,51.835525,0.916322,Spa


In [17]:
london_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Ab Kettleby,2,2,2,2,2,2
Abbas Combe,3,3,3,3,3,3
Abberley,1,1,1,1,1,1
Abberton,2,2,2,2,2,2
Abbey End,20,20,20,20,20,20
...,...,...,...,...,...,...
Acresford,1,1,1,1,1,1
Acton,1,1,1,1,1,1
Acton Bridge,1,1,1,1,1,1


In [18]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 100 uniques categories.


# Analyze each neighborhood

In [19]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighbourhood'] = london_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighbourhood,Art Gallery,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Bookstore,...,Theme Park,Trail,Train Station,Tram Station,Transportation Service,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,Abbas Combe,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Abbas Combe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbas Combe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abberley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abberton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
london_onehot.shape

(307, 101)

In [21]:
london_grouped = london_onehot.groupby('Neighbourhood').mean().reset_index()
london_grouped.head()

,Neighbourhood,Art Gallery,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Bookstore,...,Theme Park,Trail,Train Station,Tram Station,Transportation Service,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,Ab Kettleby,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abbas Combe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abberley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abberton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Abbey End,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
london_grouped.shape

(71, 101)

In [23]:
num_top_venues = 10

for hood in london_grouped['Neighbourhood']:
    #print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
import numpy as np # library to handle data in a vectorized manner

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = london_grouped['Neighbourhood']

for ind in np.arange(london_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ab Kettleby,Pub,Auto Garage,Furniture / Home Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
1,Abbas Combe,Pub,Shipping Store,Train Station,Food,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
2,Abberley,Scenic Lookout,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
3,Abberton,Spa,Gastropub,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
4,Abbey End,Hotel,Indian Restaurant,Coffee Shop,Supermarket,Mediterranean Restaurant,Furniture / Home Store,Park,Sandwich Place,Cocktail Bar,Café


In [26]:
neighbourhoods_venues_sorted.shape

(71, 11)

# Cluster neighborhoods

In [27]:
london_grouped.head()

,Neighbourhood,Art Gallery,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Bookstore,...,Theme Park,Trail,Train Station,Tram Station,Transportation Service,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,Ab Kettleby,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abbas Combe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abberley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abberton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Abbey End,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
london_grouped[['Food']]

,Food
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
66,0.0
67,0.0
68,0.0
69,0.0


In [29]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [30]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = df



In [31]:
london_merged.head()

,id,name,county,country,grid_reference,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type
0,1,Aaron's Hill,Surrey,England,SU957435,495783,143522,51.18291,-0.63098,78,GU7 2,Waverley District,South East,Suburban Area
1,2,Abbas Combe,Somerset,England,ST707226,370749,122688,51.00283,-2.41825,91,BA8 0,South Somerset District,South West,Village
2,3,Abberley,Worcestershire,England,SO744675,374477,267522,52.30522,-2.37574,152,WR6 6,Malvern Hills District,West Midlands,Village
3,4,Abberton,Essex,England,TM006190,600637,219093,51.83440,0.91066,44,CO5 7,Colchester District,Eastern,Village
5,6,Abberwick,Northumberland,England,NU129132,412938,613277,55.41325,-1.79720,93,NE66 2,Northumberland,North East,Locality


In [32]:

neighbourhoods_venues_sorted.head()


,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Ab Kettleby,Pub,Auto Garage,Furniture / Home Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
1,0,Abbas Combe,Pub,Shipping Store,Train Station,Food,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
2,1,Abberley,Scenic Lookout,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
3,4,Abberton,Spa,Gastropub,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
4,1,Abbey End,Hotel,Indian Restaurant,Coffee Shop,Supermarket,Mediterranean Restaurant,Furniture / Home Store,Park,Sandwich Place,Cocktail Bar,Café


In [33]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='name')
london_merged.dropna(inplace=True)
london_merged.shape # check the last columns!
# london_merged.head(50)
london_merged[['name','latitude','longitude','Cluster Labels']]

,name,latitude,longitude,Cluster Labels
1,Abbas Combe,51.00283,-2.41825,0.0
2,Abberley,52.30522,-2.37574,1.0
3,Abberton,51.83440,0.91066,4.0
10,Abbeydale,51.84615,-2.19922,1.0
13,Abbeydale Park,53.32291,-1.52539,0.0
...,...,...,...,...
290,Acres Nook,53.07609,-2.23685,1.0
292,Acton,53.07296,-2.55019,3.0
303,Acton Bridge,53.27354,-2.60652,2.0
304,Acton Burnell,52.61393,-2.69428,1.0


In [34]:
london_merged['Cluster Labels'] = london_merged['Cluster Labels'].astype(int)

In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['latitude'], london_merged['longitude'], london_merged['name'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Abbas Combe,370749,122688,51.00283,-2.41825,91,BA8 0,South Somerset District,South West,Village,...,Pub,Shipping Store,Train Station,Food,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
13,Abbeydale Park,431711,380770,53.32291,-1.52539,143,S17 3,Sheffield District,Yorkshire and the Humber,Suburban Area,...,Pub,Grocery Store,Cricket Ground,Hardware Store,Harbor / Marina,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market
16,Abbey Field,599500,224500,51.88336,0.89730,38,CO2 7,Colchester District,Eastern,Locality,...,Pub,Bar,Furniture / Home Store,Italian Restaurant,Food,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
29,Abbeytown,317279,550746,54.84466,-3.28970,14,CA7 4,Allerdale District,North West,Village,...,Pub,Convenience Store,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
30,Abbey Village,364054,422764,53.70007,-2.54594,161,PR6 8,Chorley District,North West,Village,...,Pub,Indian Restaurant,Outdoors & Recreation,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
35,Abbots Bromley,408370,324413,52.81719,-1.87725,119,WS15 3,East Staffordshire District,West Midlands,Village,...,Pub,Harbor / Marina,Brewery,Hardware Store,Health & Beauty Service,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
38,Abbotsford,530382,120922,50.97296,-0.14434,29,RH15 8,Mid Sussex District,South East,Hamlet,...,Pub,Pet Store,Golf Course,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
41,Abbotskerswell,285694,68785,50.50767,-3.61335,31,TQ12 5,Teignbridge District,South West,Village,...,Pub,Convenience Store,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
42,Abbots Langley,509444,201966,51.70577,-0.41754,128,WD5 0,Three Rivers District,Eastern,Village,...,Pub,Indian Restaurant,Pharmacy,Convenience Store,Chinese Restaurant,Café,Food,Dive Bar,English Restaurant,Farmers Market
44,Abbotsley,522933,256521,52.19323,-0.20294,45,PE19 6,Huntingdonshire District,Eastern,Village,...,Pub,Bus Stop,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop


In [37]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Abberley,374477,267522,52.30522,-2.37574,152,WR6 6,Malvern Hills District,West Midlands,Village,...,Scenic Lookout,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
10,Abbeydale,386373,216413,51.84615,-2.19922,47,GL4 4,Gloucester District,South West,Suburban Area,...,Playground,Pub,Indian Restaurant,Grocery Store,Food,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market
15,Abbey End,428680,271865,52.34412,-1.58043,92,CV8 1,Warwick District,West Midlands,Suburban Area,...,Hotel,Indian Restaurant,Coffee Shop,Supermarket,Mediterranean Restaurant,Furniture / Home Store,Park,Sandwich Place,Cocktail Bar,Café
17,Abbey Fields,428370,271950,52.34490,-1.58497,81,CV8 1,Warwick District,West Midlands,Suburban Area,...,Coffee Shop,Sandwich Place,Wine Bar,Hotel,Gym / Fitness Center,Indian Restaurant,Bank,Italian Restaurant,Grocery Store,Cocktail Bar
18,Abbey Gate,329500,97500,50.77278,-3.00118,55,EX13 5,East Devon District,South West,Locality,...,Home Service,Photography Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
21,Abbey Hey,389500,396500,53.46514,-2.15963,87,M18 8,Manchester District,North West,Locality,...,Stables,Home Service,Performing Arts Venue,Deli / Bodega,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
23,Abbey Hulton,390708,348651,53.03506,-2.14002,136,ST2 8,City of Stoke-on-Trent,West Midlands,Suburban Area,...,Concert Hall,Gym / Fitness Center,Grocery Store,Café,Yoga Studio,French Restaurant,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
24,Abbey Mead,504500,167500,51.39692,-0.49929,14,KT16 8,Runnymede District,South East,Locality,...,Bed & Breakfast,Yoga Studio,Garden Center,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop
25,Abbey Meads,414197,189131,51.60084,-1.79642,107,SN25 4,Swindon,South West,Suburban Area,...,Pub,Restaurant,English Restaurant,Grocery Store,Chinese Restaurant,Food,Discount Store,Dive Bar,Farmers Market,Fast Food Restaurant
28,Abbeystead,356356,454382,53.98364,-2.66703,115,LA2 9,Lancaster District,North West,Hamlet,...,Trail,Yoga Studio,Furniture / Home Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


In [38]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Abbots Ripton,523054,278110,52.38720,-0.19329,42,PE28 2,Huntingdonshire District,Eastern,Village,...,Hotel,Restaurant,Deli / Bodega,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
289,Acresford,429933,313139,52.71509,-1.55833,79,DE12 8,North West Leicestershire District,East Midlands,Village,...,Restaurant,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
303,Acton Bridge,359650,375345,53.27354,-2.60652,46,CW8 3,Cheshire West and Chester,North West,Village,...,Restaurant,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


In [39]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,Abbots Worthy,449752,132684,51.09132,-1.29097,53,SO21 1,Winchester District,South East,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
107,Aberford,443343,437258,53.82978,-1.34296,43,LS25 3,Leeds District,Yorkshire and the Humber,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
164,Abington Pigotts,530720,244481,52.08326,-0.09371,28,SG8 0,South Cambridgeshire District,Eastern,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
185,Aby,541214,378361,53.28342,0.11672,15,LN13 0,East Lindsey District,East Midlands,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
188,Acaster Malbis,458778,445447,53.90186,-1.10691,10,YO23 2,York,Yorkshire and the Humber,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
274,Ackleton,377045,298665,52.58530,-2.34023,91,WV6 7,Shropshire,West Midlands,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
281,Acomb,393128,566451,54.99260,-2.10894,79,NE46 4,Northumberland,North East,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
292,Acton,363235,353000,53.07296,-2.55019,63,CW5 8,Cheshire East,North West,Village,...,Pub,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


In [40]:
london_merged_cluster_5 = london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
london_merged_cluster_5

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Abberton,600637,219093,51.8344,0.91066,44,CO5 7,Colchester District,Eastern,Village,...,Spa,Gastropub,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [41]:
# Find the neighborhood that is the most accessible, sort by transport column and plot

In [42]:
london_grouped_busstoped = london_grouped.sort_values(by='Bus Stop',ascending=False)
london_grouped_busstoped[['Neighbourhood','Bus Stop']].head()

,Neighbourhood,Bus Stop
23,Abbots Leigh,0.500000
31,Abbotsley,0.500000
60,Acle,0.200000
70,Acton Green,0.027027
46,Abram,0.000000


In [43]:
london_grouped_busstoped = london_grouped.sort_values(by='Train Station',ascending=False)
london_grouped_busstoped[['Neighbourhood','Train Station']].head()

,Neighbourhood,Train Station
57,Acklington,0.500000
1,Abbas Combe,0.333333
32,Abbotsmead,0.250000
70,Acton Green,0.027027
67,Acton,0.000000


In [44]:
# Find the neighborhood with the most and least cafes?


In [45]:
london_merged_cluster_5[['name','latitude','longitude',
                         '1st Most Common Venue',
                         '2nd Most Common Venue',
                         '3rd Most Common Venue',
                         '4th Most Common Venue',
                         '5th Most Common Venue',
                         '6th Most Common Venue',
                         '7th Most Common Venue',
                         '8th Most Common Venue',
                         '9th Most Common Venue',
                         '10th Most Common Venue']]



,name,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Abberton,51.8344,0.91066,Spa,Gastropub,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
